Daten importieren aus CSV
------
!Pfad anpassen!

In [1]:
import pandas as pd
import numpy as np

#attribute_names = ['Age', 'Sex', 'Height', 'Weight', 'QRS duration','P-R interval']

data = pd.read_csv('Documents/2 - Entwicklung/5 - Data Science/Digethic/DS Session 2/arrhythmia.data', ',', header=None)#, na_values="?")  #, names=attribute_names)

In [2]:
data

,0,1,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
0,75,0,190,80,91,193,371,174,121,-16,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,8
1,56,1,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,6
2,54,0,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,10
3,55,0,175,94,100,202,380,179,143,28,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,1
4,75,0,190,80,88,181,360,177,103,-16,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,53,1,160,70,80,199,382,154,117,-37,...,0.0,4.3,-5.0,0.0,0.0,0.7,0.6,-4.4,-0.5,1
448,37,0,190,85,100,137,361,201,73,86,...,0.0,15.6,-1.6,0.0,0.0,0.4,2.4,38.0,62.4,10
449,36,0,166,68,108,176,365,194,116,-85,...,0.0,16.3,-28.6,0.0,0.0,1.5,1.0,-44.2,-33.2,2
450,32,1,155,55,93,106,386,218,63,54,...,-0.4,12.0,-0.7,0.0,0.0,0.5,2.4,25.0,46.6,1


In [3]:
data[278:279]

,0,1,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
278,48,1,160,61,72,204,360,155,80,40,...,0.0,11.1,-0.7,0.0,0.0,0.5,1.8,28.4,41.3,10


Datensatz vorbereiten und splitten
-----

In [4]:
data = data.replace('?',np.nan)
data[13]

0      NaN
1      NaN
2       23
3      NaN
4      NaN
      ... 
447    NaN
448    NaN
449     84
450    103
451    NaN
Name: 13, Length: 452, dtype: object

In [5]:
#data = data.fillna(0)
#performance besser, wenn nicht 0!
#besser: drop oder mittelwert
data = data.dropna()
data[13]

2       23
8       84
29     160
56     -93
71     154
      ... 
426    -84
430    -44
440    -90
449     84
450    103
Name: 13, Length: 68, dtype: object

In [155]:
#'Spalte mit Klassifizierung
y = data[279]

#Alle anderen Spalten sind Eingangsdaten
X = data.loc[:, data.columns != 279]

In [156]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

In [157]:
#print('Y Test: {}'.format(y_test.shape))
print(y_test)

158     1
388     9
89      9
376     8
188    14
356     3
85      9
93      5
297    10
252     3
237     2
256     2
88      9
370     9
359     2
285     3
403     5
207     9
366     2
430    10
319    10
450     1
424     9
Name: 279, dtype: int64


Decision Tree initialisieren, testen und darstellen
------

In [178]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=6, criterion="gini", min_samples_split=3)
dt.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=6, min_samples_split=3)

In [179]:
score = dt.score(X_test,y_test)
print('Decision Tree scores with {}% accuracy'.format(score*100))

Decision Tree scores with 26.08695652173913% accuracy


In [180]:
#from sklearn import tree
#tree.plot_tree(dt) 

In [165]:
from sklearn.tree import export_graphviz

#attribute_names = ['sepal length in cm','sepal width in cm','petal length in cm','petal width in cm', 'class']

#Export as dot file
export_graphviz(dt, out_file='arrhythmia_5_neu.dot', class_names = True) #, feature_names = attribute_names[0:4])

#Export dot to png 
from subprocess import check_call
check_call(['dot','-Tpng','arrhythmia_5_neu.dot','-o','arrhythmia_5_neu.png'])

0

Random Forest
----

In [175]:
from sklearn.model_selection import GridSearchCV

#tree parameters which shall be tested
tree_para = {'criterion':['gini','entropy'],'max_depth':[i for i in range(1,20)], 'min_samples_split':[i for i in range (2,20)]}

#GridSearchCV object
grd_clf = GridSearchCV(dt, tree_para, cv=5)

#creates differnt trees with all the differnet parameters out of our data
grd_clf.fit(X_train, y_train)

#best paramters that were found
best_parameters = grd_clf.best_params_  
print(best_parameters) 

C:\Users\Chris\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


{'criterion': 'gini', 'max_depth': 6, 'min_samples_split': 3}


In [176]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=6, criterion="gini", min_samples_split=3)
rf2 = RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)
rf2.fit(X_train,y_train)

RandomForestClassifier()

In [177]:
score = rf.score(X_test,y_test)
score2 = rf2.score(X_test,y_test)
print('Random Forest scores with {}% accuracy'.format(score*100))
print('Random Forest2 scores with {}% accuracy'.format(score2*100))

Random Forest scores with 43.47826086956522% accuracy
Random Forest2 scores with 52.17391304347826% accuracy


Akkuranz neu berechnet
------

In [181]:
from sklearn.model_selection import cross_val_score, KFold
#list with accuracies with different test and training sets of Random Forest
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

#RandomForestClassifier object
random_forest_classifier = RandomForestClassifier(n_estimators=10)
accuracies_rand_forest = cross_val_score(random_forest_classifier, X, y, cv=k_fold, n_jobs=1)

In [182]:
#new tree object with best parameters
model_with_best_tree_parameters = grd_clf.best_estimator_

#k_fold object
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

#scores reached with different splits of training/test data 
k_fold_scores = cross_val_score(model_with_best_tree_parameters, X, y, cv=k_fold, n_jobs=1)

In [183]:
import numpy as np

#scores reached with different splits of training/test data 
k_fold_scores = cross_val_score(dt, X_test, y_test, cv=k_fold, n_jobs=1)

#arithmetic mean of accuracy scores 
mean_accuracy = np.mean(k_fold_scores)

#arithmetic mean of accuracy scores 
mean_accuracy_best_parameters_tree = np.mean(k_fold_scores)

##arithmetic mean of the list with the accuracies of the Random Forest
accuracy_rand = np.mean(accuracies_rand_forest)

print('Accuracy Random Forest ' + str(round(accuracy_rand,4)))
print('Old accuracy: ' + str(round(mean_accuracy,4)))
print('Best tree accuracy: ' + str(round(mean_accuracy_best_parameters_tree,4)))

Accuracy Random Forest 0.5758
Old accuracy: 0.34
Best tree accuracy: 0.34
